In [58]:
# Initialization
import numpy as np
from numpy import linalg as la
import pandas as pd
from scipy import optimize
import warnings
import math
import sys
from tabulate import tabulate
import csv

import CovidEM
import CovidEM2
import CovidCI

import imp
imp.reload(CovidCI)
imp.reload(CovidEM) #applies changes made to file
imp.reload(CovidEM2)

# Read in data
cases = pd.read_csv('CovidStratified/Data/cases.csv', index_col=0)
cases = cases.sort_index(ascending = True)
# to retrieve entries as plain data types: cases.iloc[i,j].values
cases_dict = {}
cases_mat = cases.to_dict("split")["data"] # temp variable to hold onto arrays
for i in range(len(cases)):
    cases_dict[cases.index[i]] = cases_mat[i]
del cases_mat

prem = {
"can" : pd.read_csv('CovidStratified/Data/premCan.csv', index_col=0).to_numpy(),
"chn" : pd.read_csv('CovidStratified/Data/premChn.csv', index_col=0).to_numpy(),
"gbr" : pd.read_csv('CovidStratified/Data/premGbr.csv', index_col=0).to_numpy(),
"isr" : pd.read_csv('CovidStratified/Data/premIsr.csv', index_col=0).to_numpy(),
"ita" : pd.read_csv('CovidStratified/Data/premIta.csv', index_col=0).to_numpy(),
"nld" : pd.read_csv('CovidStratified/Data/premNLD.csv', index_col=0).to_numpy(),
}
# to retrieve entries: prem["can"]

countries = list(prem)

# print(cases)
# print(prem["can"])

In [59]:
# Example of Confidence intervals

import numpy as np               # for numerical operations
from scipy import stats          # for stats functions
from scipy import optimize as op # to maximize the likelihood

import numdifftools as nd        # to compute gradient and Hessian numerically;
                                 # the package can be found on pypi.
                                 # Another good package for that purpose
                                 # (using automatic differentiation) is autograd

from ci_rvm import find_CI
np.random.seed(451)

# Define the size of the data set
n = 100

# Define the true parameters
k, p = 5, 0.1

# Generate the data set
data = np.random.negative_binomial(k, p, size=n)

# Because the parameters are constrained to the positive range and the
# interval (0, 1), respectively, we work on a transformed parameter space
# with unbounded domain.
def transform_parameters(params):
    k, p = params
    return np.exp(k), 1/(1+np.exp(-p))
    #return k, p

# Log-Likelihood function for a negative binomial model
def LogL(params):
    k, p = transform_parameters(params)
    return stats.nbinom.logpmf(data, k, p).sum()

# negative log-Likelihood function for optimization (because we use
# minimization algorithms instead of maximization algorithms)
negLogL = lambda params: -logL(params)

# Initial guess
x0 = [4, 0.2]

# Maximize the likelihood
result = op.minimize(negLogL, x0)
#print(result.x)

# Print the result (we need to transform the parameters to the original
# parameter space to make them interpretable)
print("The estimate is: k={:5.3f}, p={:5.3f}".format(*transform_parameters(result.x)))



The estimate is: k=4.233, p=0.086


In [60]:
np.random.seed(451)

# Define gradient and Hessian
jac = nd.Gradient(LogL)
hess = nd.Hessian(LogL)

# Find confidence intervals for all parameters.
# Note: For complicated problems, it is worthwile doing this in parallel.
#       However, then we would need to encapsulate the procedure in a
#       method and define the likelihood function, gradient, and Hessian
#       on the top level of the module.
CIs = find_CI(result.x, LogL, jac, hess, alpha = 0.95,
              disp=False) # the disp argument lets the algorithm print
                         # status messages.
    
#print(result.x)
#print(CIs)
    
# CIs is a 2D numpy array with CIs[i, 0] containing the lower bound of the
# confidence interval for the i-th parameter and CIs[i, 1] containing the
# respective upper bound.

# Print the confidence intervals. Note: we need to transform the parameters
# back to the original parameter space.
original_lower = transform_parameters(CIs[:,0])
original_upper = transform_parameters(CIs[:,1])
print("Confidence interval for k: [{:5.3f}, {:5.3f}]".format(
   original_lower[0], original_upper[0]))
#print("Confidence interval for p: [{:5.3f}, {:5.3f}]".format(
#   original_lower[1], original_upper[1]))
print("\n")
print("trans: k: ", transform_parameters(result.x)[0])
print("trans: k - upper:", transform_parameters(result.x)[0] - original_upper[0])
print("trans: k - lower:", transform_parameters(result.x)[0] - original_lower[0])
print("\n")
print("non-trans: k: ", result.x[0])
print("non-trans: k - upper: ", result.x[0] - CIs[0,0])
print("non-trans: k - lower: ", result.x[0] - CIs[0,1])

Confidence interval for k: [3.106, 5.687]


trans: k:  4.2334692165135435
trans: k - upper: -1.4535306447825596
trans: k - lower: 1.1274530109756


non-trans: k:  1.4430218026703197
non-trans: k - upper:  0.30968086046292687
non-trans: k - lower:  -0.2951610412647001


In [61]:
imp.reload(CovidEM2)

starts = 1 #number of different starting points to test
trials = 1 # Number of trials per start point
iters = (starts * trials)
theta_iter = [[0] * 33] * iters
best_res = optimize.OptimizeResult(x = 100, fun = 100, success = False) #placeholder var to track best result

print ("Cumulative country calculation with randomized starting points:\n")
kwargs = dict(args = (prem, cases_dict, countries), method="L-BFGS-B")
for i in range(0, starts):
    theta_guess = np.random.gamma(4, 1/4, 14) # mean = 1, sd = 2
    for k in range(len(theta_guess)): theta_guess[k] = math.log(theta_guess[k], math.e)
    for j in range(0, trials):
        res = optimize.basinhopping(CovidEM2.Covid_KL2, theta_guess, minimizer_kwargs = kwargs)
        out_start = np.insert(np.insert(theta_guess, 2,1),10,1)
        out_end = np.insert(np.insert(res.x, 2,1),10,1)
        theta_iter[((i*trials)+j)] = np.append(np.append(out_start, out_end), res.fun)
        if (res.fun < best_res.fun): 
            best_res = res
            
    print(f"{((((i+1)*trials)/iters)*100):.2f}","%, ", end = "")


print(tabulate(theta_iter))


Cumulative country calculation with randomized starting points:

100.00 %, --------  --------  -  --------  --------  ---------  --------  --------  --------  ---------  -  ---------  --------  ---------  --------  --------  --------  ---------  -  --------  --------  --------  --------  --------  -------  --------  -  ---------  -------  --------  --------  -------  --------
0.253339  0.462547  1  -1.18867  -0.42436  -0.203693  0.276034  0.106561  -1.12905  -0.377968  1  -0.328142  0.246316  -0.172576  0.130968  0.755308  -1.61294  -0.774383  1  0.097179  0.433981  0.457346  0.496305  0.543279  1.54297  -0.91415  1  0.0149168  -11.815  0.725677  0.926855  1.21733  0.316785
--------  --------  -  --------  --------  ---------  --------  --------  --------  ---------  -  ---------  --------  ---------  --------  --------  --------  ---------  -  --------  --------  --------  --------  --------  -------  --------  -  ---------  -------  --------  --------  -------  --------


In [111]:
imp.reload(CovidEM2) #applies changes made to file
np.random.seed(451)


best_est = [0.219294308, 0.423309799, 1.124597388, 1.511734653, 1.560386081, 1.727919503, 1.734579037, 
            3.77747199,  0.71170154,  0.769347828, 0.104869899, 2.090248998, 1.98923028,  3.373799478]
for i in range(0, len(best_est)):
    best_est[i] = math.log(best_est[i])

# Initialize variables
CovidEM2.set_prem(prem)
CovidEM2.set_cases(cases_dict)
CovidEM2.set_countries(countries)


#cis_2a = CovidEM2.CI_calc2(best_res.x, CovidEM2.neg_Covid_KL2)
print("\n\n\nDone a\n\n")

cis_2b = CovidEM2.CI_calc2(best_est, CovidEM2.neg_Covid_KL2)
print("\n\n\nDone b\n\n")




Done a


iter   1<: x_0_d= 0.062; f_d= 1.921; jac_d= 0.005; nsteps= 4; x_d= 0.222; f_impr=-0.000; jac_impr= 0.550; f_e= 0.000; step= 0.125; radius= 0.444 - maximizing nuisance parameters
-> iter   1<: !!!found NEW MAXIMUM for x_0 of -0.319 (+ 0.001) at [-1.57984058 -0.80547792  0.13432221  0.46563199  0.51115072  0.53299832
  0.58268581  1.1656614  -0.53957736 -0.06606532 -2.94156228  0.52823196
  0.88188569  1.0909828 ]!!!
iter   2<: x_0_d= 0.070; f_d= 1.922; jac_d= 0.005; nsteps= 6; x_d= 0.085; f_impr=-0.001; jac_impr=-0.071; f_e= 0.000; step= 0.016; radius= 0.088 - maximizing nuisance parameters
-> iter   2<: !!!found NEW MAXIMUM for x_0 of -0.317 (+ 0.002) at [-1.58765308 -0.82375157  0.11893373  0.44971557  0.50019945  0.52522298
  0.57038656  1.47350645 -0.57520346 -0.09952441 -2.97153903  0.53303677
  0.85397846  1.11209197]!!!
iter   3<: x_0_d= 0.195; f_d= 1.923; jac_d= 0.006; nsteps= 7; x_d= 0.204; f_impr=-0.000; jac_impr=-0.265; f_e= 0.000; step= 0.250; radius= 2.809 - max

C:\Users\zakst\Documents\NIH\CovidStratified\Executables\CovidEM2.py:240: RuntimeWarning: overflow encountered in exp
  params[i] = np.exp(params_in[i])


Traceback (most recent call last):
  File "C:\Users\zakst\anaconda3\lib\site-packages\ci_rvm\ci_rvm.py", line 805, in find_CI_bound
    precise, fActual, JActual = test_precision()
  File "C:\Users\zakst\anaconda3\lib\site-packages\ci_rvm\ci_rvm.py", line 372, in test_precision
    fActual = fun(xTmp)
  File "C:\Users\zakst\anaconda3\lib\site-packages\ci_rvm\ci_rvm.py", line 146, in __call__
    return self.fun(*args, **kwargs)
  File "C:\Users\zakst\anaconda3\lib\site-packages\ci_rvm\ci_rvm.py", line 246, in <lambda>
    fun = lambda x: fun2(flip(x))
  File "C:\Users\zakst\Documents\NIH\CovidStratified\Executables\CovidEM2.py", line 294, in neg_Covid_KL2
    return -1 * Covid_KL2(theta0, prem_global, cases_global, countries_global)
  File "C:\Users\zakst\Documents\NIH\CovidStratified\Executables\CovidEM2.py", line 255, in Covid_KL2
    res = Covid_KL_k2(theta0, prem_in[country_codes[i]], cases_in[country_codes[i]])
  File "C:\Users\zakst\Documents\NIH\CovidStratified\Executables\Covid

C:\Users\zakst\Documents\NIH\CovidStratified\Executables\CovidEM2.py:240: RuntimeWarning: overflow encountered in exp
  params[i] = np.exp(params_in[i])


Traceback (most recent call last):
  File "C:\Users\zakst\anaconda3\lib\site-packages\ci_rvm\ci_rvm.py", line 805, in find_CI_bound
    precise, fActual, JActual = test_precision()
  File "C:\Users\zakst\anaconda3\lib\site-packages\ci_rvm\ci_rvm.py", line 372, in test_precision
    fActual = fun(xTmp)
  File "C:\Users\zakst\anaconda3\lib\site-packages\ci_rvm\ci_rvm.py", line 146, in __call__
    return self.fun(*args, **kwargs)
  File "C:\Users\zakst\Documents\NIH\CovidStratified\Executables\CovidEM2.py", line 294, in neg_Covid_KL2
    return -1 * Covid_KL2(theta0, prem_global, cases_global, countries_global)
  File "C:\Users\zakst\Documents\NIH\CovidStratified\Executables\CovidEM2.py", line 255, in Covid_KL2
    res = Covid_KL_k2(theta0, prem_in[country_codes[i]], cases_in[country_codes[i]])
  File "C:\Users\zakst\Documents\NIH\CovidStratified\Executables\CovidEM2.py", line 227, in Covid_KL_k2
    u_tilda = scv_eig2(s = theta0[0:(int(len(theta0)/2))], c = prem_in, v = theta0[(int(len(

...

C:\Users\zakst\Documents\NIH\CovidStratified\Executables\CovidEM2.py:173: UserWarning: Eigenvector contains both negative and positive elements
  warnings.warn("Eigenvector contains both negative and positive elements")


Traceback (most recent call last):
  File "C:\Users\zakst\anaconda3\lib\site-packages\ci_rvm\ci_rvm.py", line 805, in find_CI_bound
    precise, fActual, JActual = test_precision()
  File "C:\Users\zakst\anaconda3\lib\site-packages\ci_rvm\ci_rvm.py", line 372, in test_precision
    fActual = fun(xTmp)
  File "C:\Users\zakst\anaconda3\lib\site-packages\ci_rvm\ci_rvm.py", line 146, in __call__
    return self.fun(*args, **kwargs)
  File "C:\Users\zakst\Documents\NIH\CovidStratified\Executables\CovidEM2.py", line 294, in neg_Covid_KL2
    return -1 * Covid_KL2(theta0, prem_global, cases_global, countries_global)
  File "C:\Users\zakst\Documents\NIH\CovidStratified\Executables\CovidEM2.py", line 255, in Covid_KL2
    res = Covid_KL_k2(theta0, prem_in[country_codes[i]], cases_in[country_codes[i]])
  File "C:\Users\zakst\Documents\NIH\CovidStratified\Executables\CovidEM2.py", line 227, in Covid_KL_k2
    u_tilda = scv_eig2(s = theta0[0:(int(len(theta0)/2))], c = prem_in, v = theta0[(int(len(

C:\Users\zakst\Documents\NIH\CovidStratified\Executables\CovidEM2.py:240: RuntimeWarning: overflow encountered in exp
  params[i] = np.exp(params_in[i])
C:\Users\zakst\Documents\NIH\CovidStratified\Executables\CovidEM2.py:140: RuntimeWarning: invalid value encountered in matmul
  builder = np.matmul(np.matmul(s,c),v)


Traceback (most recent call last):
  File "C:\Users\zakst\anaconda3\lib\site-packages\ci_rvm\ci_rvm.py", line 805, in find_CI_bound
    precise, fActual, JActual = test_precision()
  File "C:\Users\zakst\anaconda3\lib\site-packages\ci_rvm\ci_rvm.py", line 372, in test_precision
    fActual = fun(xTmp)
  File "C:\Users\zakst\anaconda3\lib\site-packages\ci_rvm\ci_rvm.py", line 146, in __call__
    return self.fun(*args, **kwargs)
  File "C:\Users\zakst\anaconda3\lib\site-packages\ci_rvm\ci_rvm.py", line 246, in <lambda>
    fun = lambda x: fun2(flip(x))
  File "C:\Users\zakst\Documents\NIH\CovidStratified\Executables\CovidEM2.py", line 294, in neg_Covid_KL2
    return -1 * Covid_KL2(theta0, prem_global, cases_global, countries_global)
  File "C:\Users\zakst\Documents\NIH\CovidStratified\Executables\CovidEM2.py", line 255, in Covid_KL2
    res = Covid_KL_k2(theta0, prem_in[country_codes[i]], cases_in[country_codes[i]])
  File "C:\Users\zakst\Documents\NIH\CovidStratified\Executables\Covid

C:\Users\zakst\Documents\NIH\CovidStratified\Executables\CovidEM2.py:173: UserWarning: Eigenvector contains both negative and positive elements
  warnings.warn("Eigenvector contains both negative and positive elements")


In [112]:
imp.reload(CovidEM2) #applies changes made to file
np.random.seed(451)

bad_params = [-1.72245253, -0.91245638,  0.01888201,  0.34871368,  0.35877798,
              0.39279521,  0.41702544,  1.95818931, -0.10753684,  0.3775735 ,
              -9.22278978,  1.15643563,  1.2910407 ,  1.64917554]

CovidEM2.Covid_KL2(bad_params, prem, cases_dict, countries)

res = optimize.basinhopping(CovidEM2.Covid_KL2, bad_params, minimizer_kwargs = kwargs)

In [113]:
print(res)

                        fun: 0.3167849972770691
 lowest_optimization_result:       fun: 0.3167849972770691
 hess_inv: <14x14 LbfgsInvHessProduct with dtype=float64>
      jac: array([ 5.55111516e-06,  5.00710582e-06, -6.49480470e-07, -1.16684440e-05,
        4.81281681e-06, -2.31481501e-06,  3.68594042e-06, -4.62407893e-06,
        1.40998323e-06, -1.06747944e-05, -1.11022293e-08, -3.96349618e-06,
       -1.32116539e-06,  7.93809467e-06])
  message: 'CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH'
     nfev: 1635
      nit: 99
     njev: 109
   status: 0
  success: True
        x: array([ -1.61258473,  -0.7744435 ,   0.09740116,   0.4341267 ,
         0.4574897 ,   0.49643494,   0.54340821,   1.54197721,
        -0.91237976,   0.01405923, -13.1434035 ,   0.72507274,
         0.92642529,   1.21675698])
                    message: ['requested number of basinhopping iterations completed successfully']
      minimization_failures: 0
                       nfev: 151485
                   

In [114]:
thetas = best_est
confint = cis_2b

print("Overhauled KL, Estimate")
print("non-trans: theta: ", thetas[0])
print("non-trans: CIs:", confint[0,:])
print("non-trans: theta - upper: ", thetas[0] - confint[0,0])
print("non-trans: theta - lower: ", thetas[0] - confint[0,1])
print("\n")

for i in range(0,len(thetas)):
    print("Parameter #", i+1)
    print("trans: theta: ", CovidEM2.param_trans2(thetas)[i])
    print("trans: CIs:", CovidEM2.param_trans2(confint[i,:]))
    print("trans: theta - upper:", CovidEM2.param_trans2(thetas)[i] - CovidEM2.param_trans2(confint[i,0]))
    print("trans: theta - lower:", CovidEM2.param_trans2(thetas)[i] - CovidEM2.param_trans2(confint[i,1]))
    print("\n")

Overhauled KL, Estimate
non-trans: theta:  -1.517340579207497
non-trans: CIs: [-1.75830738 -1.3460149 ]
non-trans: theta - upper:  0.240966796875
non-trans: theta - lower:  -0.17132568359375


Parameter # 1
trans: theta:  0.219294308
trans: CIs: [0.17233631766572943, 0.2602754213875373]
trans: theta - upper: 0.04695799033427056
trans: theta - lower: -0.040981113387537293


Parameter # 2
trans: theta:  0.423309799
trans: CIs: [0.3537965293739492, 0.6038150065919994]
trans: theta - upper: 0.06951326962605081
trans: theta - lower: -0.1805052075919994


Parameter # 3
trans: theta:  1.124597388
trans: CIs: [1.0342586189947878, 1.2918188582038286]
trans: theta - upper: 0.0903387690052122
trans: theta - lower: -0.1672214702038286


Parameter # 4
trans: theta:  1.511734653
trans: CIs: [1.3934404525236117, 1.787712020316659]
trans: theta - upper: 0.11829420047638828
trans: theta - lower: -0.275977367316659


Parameter # 5
trans: theta:  1.560386081
trans: CIs: [1.4334647434051875, 1.84563932395

In [ ]:
ci_header = ["Age", "Lower", "Estimate", "Upper"]

ci_res = [["s0-9","s10-19","s30-39","s40-49","s50-59","s60-69","s70-79",
           "v0-9","v10-19","v30-39","v40-49","v50-59","v60-69","v70-79"],
          CovidEM2.param_trans2(cis_2b[:,0]),
          CovidEM2.param_trans2(best_est),
          CovidEM2.param_trans2(cis_2b[:,1])]
print(best_est - cis_2b[:,0])
print(best_est)
print(best_est - cis_2b[:,1])
ci_res = np.array(ci_res).T.tolist()

print(ci_res)
